In [1]:
import cv2
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
def load_and_preprocess_images(clear_skin, pimples_images, target_size=(224, 224)):
    # Initialize lists to store preprocessed images and labels (0 for clear skin, 1 for pimples)
    images = []
    labels = []

    # Load and preprocess images with clear skin
    for filename in os.listdir(clear_skin):
        if filename.endswith(".jpg"):
            image_path = os.path.join(clear_skin, filename)
            image = cv2.imread(image_path)
            image = cv2.resize(image, target_size)
            image = image.astype("float") / 255.0
            images.append(image)
            labels.append(0)  # 0 indicates clear skin

    # Load and preprocess images with pimples/acne
    for filename in os.listdir(pimples_images):
        if filename.endswith(".jpg"):
            image_path = os.path.join(pimples_images, filename)
            image = cv2.imread(image_path)
            image = cv2.resize(image, target_size)
            image = image.astype("float") / 255.0
            images.append(image)
            labels.append(1)  # 1 indicates presence of pimples/acne

    # Convert the lists to NumPy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Specify the paths to your clear skin and pimples image folders
clear_skin_folder = "clear_skin"
pimples_folder = "pimples_images"

# Load and preprocess images from both folders
images, labels = load_and_preprocess_images(clear_skin_folder, pimples_folder)

# Now, 'images' contains the preprocessed images, and 'labels' contains the corresponding labels.


In [ ]:
pimples_folder.info()

In [3]:
# Define the CNN architecture
def create_cnn_model(input_shape):
    model = keras.Sequential()

    # Convolutional layer 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Convolutional layer 3 (optional)
    # You can add more convolutional layers for deeper feature extraction

    # Flatten the output for the fully connected layers
    model.add(layers.Flatten())

    # Fully connected layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))  # Dropout for regularization
    model.add(layers.Dense(1, activation='sigmoid'))  # 1 output node for binary classification

    return model

# Specify the input shape (target size and number of color channels)
input_shape = (224, 224, 3)  # Adjust according to your preprocessing target size and color channels

# Create the CNN model
model = create_cnn_model(input_shape)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [4]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")



Epoch 1/10
64/64 [==============================] - 288s 4s/step - loss: 0.4883 - accuracy: 0.9360 - val_loss: 0.7514 - val_accuracy: 0.8965
Epoch 2/10
64/64 [==============================] - 297s 5s/step - loss: 0.0995 - accuracy: 0.9775 - val_loss: 0.0849 - val_accuracy: 0.9766
Epoch 3/10
64/64 [==============================] - 304s 5s/step - loss: 0.0621 - accuracy: 0.9805 - val_loss: 0.0573 - val_accuracy: 0.9785
Epoch 4/10
64/64 [==============================] - 206s 3s/step - loss: 0.0480 - accuracy: 0.9854 - val_loss: 0.0952 - val_accuracy: 0.9746
Epoch 5/10
64/64 [==============================] - 199s 3s/step - loss: 0.0486 - accuracy: 0.9844 - val_loss: 0.0536 - val_accuracy: 0.9805
Epoch 6/10
64/64 [==============================] - 204s 3s/step - loss: 0.0528 - accuracy: 0.9824 - val_loss: 0.0671 - val_accuracy: 0.9805
Epoch 7/10
64/64 [==============================] - 206s 3s/step - loss: 0.0289 - accuracy: 0.9941 - val_loss: 0.0630 - val_accuracy: 0.9805
Epoch 8/10
64

In [5]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Convert predicted probabilities to binary labels (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

# Print classification report and confusion matrix
print(classification_report(y_test, y_pred_binary))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_binary))

21/21 [==============================] - 40s 867ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       485
           1       0.97      0.97      0.97       156

    accuracy                           0.98       641
   macro avg       0.98      0.98      0.98       641
weighted avg       0.98      0.98      0.98       641

Confusion Matrix:
[[480   5]
 [  5 151]]


In [7]:
#############################################################Testing############################################################## 

import cv2
import numpy as np

# Load and preprocess a sample image (replace 'sample.jpg' with the actual image filename)
image_path = 'sample.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (224, 224))  # Resize to match the input size used during training
image = image.astype("float") / 255.0

# Make a prediction on the sample image
predictions = model.predict(np.expand_dims(image, axis=0))

# Interpret the prediction
predicted_class = 1 if predictions[0][0] > 0.5 else 0

if predicted_class == 0:
    label = "Clear Skin"
else:
    label = "Pimples/Acne"

print(f"Predicted Label: {label}")


1/1 [==============================] - 0s 118ms/step
Predicted Label: Pimples/Acne


In [ ]:
#########################################################Predicting################################################

# Load the face detection classifier (Haar Cascade or any other)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the camera
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # Loop through detected faces
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face = frame[y:y + h, x:x + w]

        # Preprocess the face image for your model
        face = cv2.resize(face, (224, 224))
        face = face.astype("float") / 255.0
        face = np.expand_dims(face, axis=0)

        # Make a prediction using your model
        predictions = model.predict(face)

        # Interpret the prediction
        if predictions[0][0] > 0.5:
            label = "Pimples/Acne"
            color = (0, 0, 255)  # Red for pimples
        else:
            label = "Clear Skin"
            color = (0, 255, 0)  # Green for clear skin

        # Draw a bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Display the frame with bounding boxes and labels
    cv2.imshow('Facial Health Detection', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
cap.release()
cv2.destroyAllWindows()


In [15]:


# Load the trained model (replace 'your_model.h5' with the actual filename)
#model = keras.models.load_model('your_model.h5')

# Load the face detection classifier (Haar Cascade or any other)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the camera
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # Loop through detected faces
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face = frame[y:y + h, x:x + w]

        # Preprocess the face image for your model
        face = cv2.resize(face, (224, 224))
        face = face.astype("float") / 255.0
        face = np.expand_dims(face, axis=0)

        # Make a prediction using your model
        predictions = model.predict(face)

        # Interpret the prediction
        if predictions[0][0] > 0.5:
            label = "Pimples/Acne"
            color = (0, 0, 255)  # Red for pimples
        else:
            label = "Clear Skin"
            color = (0, 255, 0)  # Green for clear skin

        # Draw a bounding box and label on the frame only if pimples are detected
        if label == "Pimples/Acne":
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Display the frame with bounding boxes and labels
    cv2.imshow('Facial Health Detection', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
cap.release()
cv2.destroyAllWindows()


In [6]:
###############################################capture using droid cam#######################################################
# Load the face detection classifier (Haar Cascade or any other)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Specify the IP address and port of your DroidCam camera
droidcam_ip = "10.237.241.176"
droidcam_port = "4747"

# Initialize the camera using the DroidCam IP and port
cap = cv2.VideoCapture(f"http://{droidcam_ip}:{droidcam_port}/video")

while True:
    # Capture a frame from the external camera
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # Loop through detected faces
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face = frame[y:y + h, x:x + w]

        # Preprocess the face image for your model
        face = cv2.resize(face, (224, 224))
        face = face.astype("float") / 255.0
        face = np.expand_dims(face, axis=0)

        # Make a prediction using your model
        predictions = model.predict(face)

        # Interpret the prediction
        if predictions[0][0] > 0.5:
            label = "Pimples/Acne"
            color = (0, 0, 255)  # Red for pimples
        else:
            label = "Clear Skin"
            color = (0, 255, 0)  # Green for clear skin

        # Draw a bounding box and label on the frame only if pimples are detected
        if label == "Pimples/Acne":
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)


    # Display the frame with bounding boxes and labels
    cv2.imshow('Facial Health Detection', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [6]:


# Load the face detection classifier (Haar Cascade or any other)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the camera
cap = cv2.VideoCapture(1)  # Use default camera (should automatically use the USB camera)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    if not ret:
        break

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

   # Loop through detected faces
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face = frame[y:y + h, x:x + w]

        # Preprocess the face image for your model
        face = cv2.resize(face, (224, 224))
        face = face.astype("float") / 255.0
        face = np.expand_dims(face, axis=0)

        # Make a prediction using your model
        predictions = model.predict(face)

        # Interpret the prediction
        if predictions[0][0] > 0.5:
            label = "Pimples/Acne"
            color = (0, 0, 255)  # Red for pimples
        else:
            label = "Clear Skin"
            color = (0, 255, 0)  # Green for clear skin

        # Draw a bounding box and label on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Display the frame with bounding boxes and labels
    cv2.imshow('Facial Health Detection', frame)

    # Exit the loop when the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the OpenCV window
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 63ms/step
